In [35]:
import json
import numpy as np
import pandas as pd
from app import dataset_word2vec, dataset_tfid
from sklearn.metrics import precision_score, recall_score
from app.datasets import dataset_ratings_user
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV, RidgeCV, BayesianRidge, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error, precision_score, recall_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

In [36]:
def evaluate(X, user, index, df_ratings):
    df_movies_u = dataset_ratings_user(X,df_ratings=df_ratings, user=user) 
    #train, test = train_test_split(df_movies_u , test_size=0.2)
    train = df_movies_u.loc[index[str(user)]['train'], :]
    test = df_movies_u.loc[index[str(user)]['test'], :]
    X_train = train.drop(columns=['rating_user','title'])
    y_train = train['rating_user']
    X_test = test.drop(columns=['rating_user','title'])
    y_test = test['rating_user']
    df_movies_u = None
    return train_predict(X_train, y_train, X_test, y_test)




def train_predict(X_train, y_train, X_test, y_test):

    precision = {}
    recall = {}
    y_true = (y_test >= 3) * 1

    def get_precision_recall(name, model):
        model = model.fit(X_train, y_train)
        y_pred = (model.predict(X_test) >= 3)*1
        precision[name] = precision_score(y_true, y_pred)
        recall[name] = recall_score(y_true, y_pred)
    ridge = RidgeCV(cv=5)
    get_precision_recall('ridge', GridSearchCV(Ridge(), {'alpha':[1e-3, 1e-2, 1e-1, 1]}))
    #get_precision_recall('lasso', GridSearchCV(Lasso(), {'alpha':[1e-3, 1e-2, 1e-1, 1]}))
    get_precision_recall('svr', GridSearchCV(SVR(),{'kernel':('linear', 'rbf'), 'C':[1, 10]}))
    get_precision_recall('elastic', GridSearchCV(ElasticNet(), {'alpha':[1e-3, 1e-2, 1e-1, 1]}))
    #get_precision_recall('sgd', GridSearchCV(SGDRegressor(), {'alpha':[1e-3, 1e-2, 1e-1, 1]}))
    #get_precision_recall('ada',AdaBoostRegressor(random_state=0, n_estimators=100))
    #get_precision_recall('gbr',GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0, loss='ls'))
    return precision, recall


def evaluate_method1(X, users, index, df_ratings):
    precisions, recalls = [], []
    columns = ['ridge', 'lasso', 'elastic', 'svr', 'sgd']
    i = 0
    for user in users:
        precision, recall = evaluate(X, int(user), index, df_ratings)
        precisions.append(precision)
        recalls.append(recall)
        print('i:', i)
        i += 1
        
    df_precisions = pd.DataFrame(precisions, index=users, columns=columns)
    df_recalls = pd.DataFrame(recalls, index=users, columns=columns)
    return df_precisions, df_recalls

In [25]:
index = {}
with open('./app/datasets/index_sample.txt') as json_file:
    index = json.load(json_file)
len(index.keys())


101

In [3]:
df_ratings = pd.read_table('./app/datasets/ml-1m/ratings.dat', delimiter='::', names=['userId', 'movieId', 'rating', 'timestamp'], engine='python')

In [4]:
new_feats = None

In [5]:
df_movies, new_feats = dataset_tfid(['genres', 'rating', 'runtimes', 'year'], op='sum', n_features=200, n_words=2000)

In [26]:
del index['last_user']

In [22]:
import warnings
warnings.filterwarnings('ignore')

In [37]:
df_precisions, df_recalls = evaluate_method1(df_movies, index.keys(), index, df_ratings)

i: 0
i: 1
i: 2
i: 3
i: 4
i: 5
i: 6
i: 7
i: 8
i: 9
i: 10
i: 11
i: 12
i: 13
i: 14
i: 15
i: 16
i: 17
i: 18
i: 19
i: 20
i: 21
i: 22
i: 23
i: 24
i: 25
i: 26
i: 27
i: 28
i: 29
i: 30
i: 31
i: 32
i: 33
i: 34
i: 35
i: 36
i: 37
i: 38
i: 39
i: 40
i: 41
i: 42
i: 43
i: 44
i: 45
i: 46
i: 47
i: 48
i: 49
i: 50
i: 51
i: 52
i: 53
i: 54
i: 55
i: 56
i: 57
i: 58
i: 59
i: 60
i: 61
i: 62
i: 63
i: 64
i: 65
i: 66
i: 67
i: 68
i: 69
i: 70
i: 71
i: 72
i: 73
i: 74
i: 75
i: 76
i: 77
i: 78
i: 79
i: 80
i: 81
i: 82
i: 83
i: 84
i: 85
i: 86
i: 87
i: 88
i: 89
i: 90
i: 91
i: 92
i: 93
i: 94
i: 95
i: 96
i: 97
i: 98
i: 99


In [38]:
df_precisions.mean()

ridge      0.886170
lasso           NaN
elastic    0.878590
svr        0.867956
sgd             NaN
dtype: float64

In [39]:
df_recalls.mean()

ridge      0.877878
lasso           NaN
elastic    0.907704
svr        0.885604
sgd             NaN
dtype: float64